In [28]:
import sys
import os
from os import getenv
from os import path
import multiprocessing

In [29]:
target_traces_dir = '/path/to/traces' # traces directory has *.traceg files

In [30]:
traceg_file_list = []
for file in os.listdir(target_traces_dir):
    if file.endswith(".traceg"):
        traceg_file_list.append(path.join(target_traces_dir, file))

In [31]:
target_trace_file_list = \
    [x for x in traceg_file_list]

In [32]:
def parse_inst(line):
    data = line.strip().split()
    item = {}
    item['pc'] = int(data[0], 16)
    
    item['thread_mask'] = bin(int(data[1], 16))[2:]
    
    item['num_dest'] = int(data[2])
    num_dest = int(data[2])
    item['dest_regs'] = data[3 : 3 + num_dest]
    
    item['opcode_tokens'] = data[3 + num_dest].split('.')
    
    num_src_idx = 3 + num_dest + 1
    item['num_src'] = int(data[num_src_idx])
    num_src = int(data[num_src_idx])
    item['src_regs'] = data[num_src_idx + 1 : num_src_idx + 1 + num_src]
    
    mem_width_idx = num_src_idx + 1 + num_src
    item['mem_width'] = int(data[mem_width_idx])
    
    if len(data) > mem_width_idx + 1:
        item['addr_compress_mode'] = int(data[mem_width_idx + 1])
        item['addr_list'] = data[mem_width_idx + 2 : ]
    
    return item

In [33]:
def get_non_stride_mem_addr_list(num_active_threads, base_addr_list, access_mode):
    assert(len(base_addr_list) == num_active_threads)
    return base_addr_list

In [34]:
def get_stride_mem_addr_list(num_active_threads, base_addr_list, access_mode):
    addr_list = []
    base_addr = base_addr_list[0]
    stride = base_addr_list[1]
    
    addr_list.append(base_addr)
    for i in range(num_active_threads - 1):
        base_addr += stride
        addr_list.append(base_addr)
    
    assert(len(addr_list) == num_active_threads)
    return addr_list

In [35]:
def get_base_delta_mem_addr_list(num_active_threads, base_addr_list, access_mode):
    addr_list = []
    base_addr = base_addr_list[0]
    addr_list.append(base_addr)
    # stride is same as the first delta
    if len(base_addr_list) == 1:
        assert(len(addr_list) == num_active_threads)
        return addr_list
    else:
        for next_stride in base_addr_list[1 : ]:
            base_addr += next_stride
            addr_list.append(base_addr)

        assert(len(addr_list) == num_active_threads)
        return addr_list

In [36]:
def get_mem_addr_list(num_active_threads, base_addr_list, access_mode):
    if access_mode == 0:
        base_addr_list = [int(x, 16) for x in base_addr_list]
        return get_non_stride_mem_addr_list(num_active_threads, base_addr_list, access_mode)
    # stride
    elif access_mode == 1:
        base_addr_list[0] = int(base_addr_list[0], 16)
        base_addr_list[1:] = [int(x) for x in base_addr_list[1:]]
        return get_stride_mem_addr_list(num_active_threads, base_addr_list, access_mode)
    # base delta
    else:
        base_addr_list[0] = int(base_addr_list[0], 16)
        base_addr_list[1:] = [int(x) for x in base_addr_list[1:]]
        return get_base_delta_mem_addr_list(num_active_threads, base_addr_list, access_mode)

In [37]:
TARGET_OPCODE = ['LDG', 'STG', 'LD', 'ST', 'ATOMG', 'ATOM', 'RED']
def generate_pagetable(filepath):
    PTE = set()
    with open(filepath, 'r') as r_file:
        start_inst = False
        start_warp = False
        for line in r_file:
            if 'insts =' in line:
                start_inst = True
            elif 'warp =' in line:
                start_warp = True
            elif len(line.strip().split()) == 0: # empty line
                start_inst, start_warp = False, False
            elif start_inst and start_warp:
                inst = parse_inst(line)
                if inst['thread_mask'].count('1') > 0:
                    opcode = inst['opcode_tokens'][0]
                    num_active_threads = inst['thread_mask'].count('1')
                    addr_list = []
                    if opcode in TARGET_OPCODE:
                        addr_list = \
                            get_mem_addr_list(num_active_threads, inst['addr_list'], inst['addr_compress_mode'])
                        for addr in addr_list:
                            # Assume 4KB page
                            page_num = addr >> 12
                            PTE.add(page_num)
    
    return PTE

In [38]:
p = multiprocessing.Pool(24)
PTE_list = p.map(generate_pagetable, target_trace_file_list)

In [39]:
PTE_all = set()

for PTE in PTE_list:
    PTE_all |= PTE

In [40]:
num_pages_path = path.join(target_traces_dir, 'num_pages.txt')

num_pages = len(PTE_all)
with open(num_pages_path, 'w') as w_file:
    w_file.write('{} {}\n'.format(num_pages, num_pages))